In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
%matplotlib inline
%load_ext qat.core.magic
#QPU connection
QLMASS = True
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg()    

In [ ]:
import copy
import scipy.optimize as so

In [ ]:
from QuantumMultiplexors_Module_examples import expectation_loading_data
from PhaseAmplification_Module import load_q_gate

In [ ]:
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [ ]:
#number of Qbits for the circuit
n_qbits = 6
#The number of bins 
m_bins = 2**n_qbits
LowerLimit = 0.0
UpperLimit = 1.0 

X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
f_X = f(X)

## Circuit

In [ ]:
Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
Q_Gate = load_q_gate(P_Gate, R_gate)

## Maximum Likelihood

For this case we start with the loading data using $\mathcal{P}$ and $\mathcal{R}$ gates. Now we do following steps:

* 1. Apply $\hat{Q}^{m_k}$ on the circuit. With $k=0$ $m_0=1$
* 2. Do $N_k$ measurements of the last qbit. Imagine we get $h_k$ the state $|\Psi_1\rangle$ ($N_k-h_k$ times state  $|\Psi_0\rangle$).
* 3. We calculate the likelihood of the before measurements:

$$L_{k}(h_k/\theta) = (\sin^2[(2*m_k+1)\theta])^{h_k}(\cos^2[(2*m_k+1)\theta])^{N_k-h_k}$$
* 4. We can calculate now the total Likelihood for a vector $\mathbf{h}=(h_0, h_1,...,h_M)$:

$$L(\mathbf{h}/\theta) = \prod_{k=0}^{M}{L_{k}(h_k/\theta)}$$
* 5. We get $\theta_{a}$ that maximizes $L(\mathbf{h}/\theta)$. In general is better to opitimize de logarithm of the $L(\mathbf{h}/\theta)$:

$$\theta_{a} = arg \ max \ln{L(\mathbf{h}/\theta)}= arg \ max \sum_{k=0}^{M} \Big( 2h_k\ln(\sin[(2*m_k+1)\theta])+2(N_k-h_k)\ln(\cos[(2*m_k+1)\theta]) \Big)$$
* 6. Go to first step with $k=1$ and $m_1=1$

In [ ]:
def Do(q_prog, q_gate, n_ops):
    q_bits = q_prog.registers[0]
    for i in range(n_ops):
        q_prog.apply(q_gate, q_bits)
    circuit = q_prog.to_circ(submatrices_only=True)
    job = circuit.to_job(qubits=[len(q_bits)-1])#, nbshots = nbshots)
    return job, circuit

In [ ]:
def likelihood(theta, m_k, h_k, n_k):
    theta_ = (2*m_k+1)*theta
    first_term = 2*h_k*np.log(np.abs(np.sin(theta_)))
    second_term = 2*(n_k-h_k)*np.log(np.abs(np.cos(theta_)))
    l_k = first_term + second_term
    return -np.sum(l_k)

def launch_likelihood(theta, pdf):
    h_k = pdf['h_k_shots']
    m_k = pdf['m_k']   
    n_k = pdf['shots']
    return likelihood(theta, m_k, h_k, n_k)

In [ ]:
def mlae(q_prog, q_gate, k, lineal_qpu, nbshots = 0):
    
    list_h_k = []
    list_of_circuits = []

    for m_k in range(k):
        step_job, step_cricuit = Do(copy.deepcopy(q_prog), q_gate, m_k)#, nbshots)
        list_of_circuits.append(step_cricuit)
        if nbshots != 0:
            step_job.nbshots = nbshots
        step_result = run_job(lineal_qpu.submit(step_job))
        step_pdf = postprocess_results(step_result)
        try:
            h_k = step_pdf[step_pdf['States'] == '|1>']['Probability'].values[0]
        except IndexError:
            #Can happen that P|1> = 0. Then the pdf do not have the state. In this case h_k = 0
            h_k = 0
        list_h_k.append([m_k, h_k])  
        
    if nbshots == 0:
        number_of_shots = 100
    else:
        number_of_shots = nbshots
    #Probability for  |1>
    p_1 = pd.DataFrame(list_h_k, columns= ['m_k', 'h_k'])
    p_1['h_k_shots'] = round(p_1['h_k']*number_of_shots, 0).astype(int)
    p_1['shots'] = number_of_shots  
    
    r = so.brute(
        likelihood,
        [(0+delta, 0.5*np.pi-delta)],
        (p_1['m_k'], p_1['h_k_shots'], p_1['shots']),
        50,
        disp=True
    )    

In [ ]:
optimizer = so.brute

In [ ]:
so.brute?

In [ ]:
K=10
nbshots = 10000
list_h_k = []
list_of_circuits = []

for m_k in range(K):
    step_job, step_cricuit = Do(copy.deepcopy(Qprog), Q_Gate, m_k)#, nbshots)
    list_of_circuits.append(step_cricuit)
    if nbshots != 0:
        step_job.nbshots = nbshots
    step_result = run_job(lineal_qpu.submit(step_job))
    step_pdf = postprocess_results(step_result)
    try:
        h_k = step_pdf[step_pdf['States'] == '|1>']['Probability'].values[0]
    except IndexError:
        #Can happen that P|1> = 0. Then the pdf do not have the state. In this case h_k = 0
        h_k = 0
    list_h_k.append([m_k, h_k])       

In [ ]:
if nbshots == 0:
    number_of_shots = 100
else:
    number_of_shots = nbshots
#Probabilidad estado |1>
p_1 = pd.DataFrame(list_h_k, columns= ['m_k', 'h_k'])
p_1['h_k_shots'] = round(p_1['h_k']*number_of_shots, 0).astype(int)
p_1['shots'] = number_of_shots

In [ ]:
p_1

In [ ]:
delta = 0.0001
theta = np.linspace(0+delta, 0.5*np.pi-delta)
y = [launch_likelihood(t, p_1) for t in theta]
plt.plot(theta,y)

In [ ]:
r = so.brute(
    likelihood,
    [(0+delta, 0.5*np.pi-delta)],
    (p_1['m_k'], p_1['h_k_shots'], p_1['shots']),
    50,
    disp=True
)

In [ ]:
r[0]

In [ ]:
def get_sub_pdf(input_pdf, state):
    pdf = input_pdf.copy(deep=True)
    phi = pdf[pdf['States'] == state]
    phi.reset_index(drop=True, inplace=True)
    phi.drop(columns=['Amplitude', 'States'], inplace = True)
    phi.rename(
        columns = {'Probability': 'Probability_{}'.format(state)},
        inplace = True
    )
    return phi.drop_duplicates()
    return phi

In [ ]:
nbshots = 0
list_h_k = []
m_k = 0

step_job, step_cricuit = Do(copy.deepcopy(Qprog), Q_Gate, m_k)#, nbshots)
#ListOfCircutis.append(step_cricuit)
if nbshots != 0:
    step_job.nbshots = nbshots
step_result = run_job(lineal_qpu.submit(step_job))
step_pdf = postprocess_results(step_result)
#ListOfPdfs.append(step_pdf)
try:
    h_k = step_pdf[step_pdf['States'] == '|1>']['Probability'].values[0]
except IndexError:
    #Can happen that P|1> = 0. Then the pdf do not have the state. In this case h_k = 0
    h_k = 0
list_h_k.append([m_k, h_k])  

In [ ]:
list_h_k

In [ ]:
p_1